In [0]:
#Notes
#Columns needed from bronze - no Rally column because it is null
#match_id, SetNo, GameNo, PointNumber, PointWinner, RallyCount, SetWinner(to determine match winner)

In [0]:
from pyspark.sql.functions import col

catalog_name = 'workspace'
schema_name = 'bronze'
table_name = 'raw_wimbledon_points_2023'
columns = ['match_id', 'SetNo', 'GameNo', 'PointNumber', 'PointWinner', 'RallyCount', 'SetWinner']
select_columns = ', '.join(columns)

query = f"SELECT {select_columns} FROM {catalog_name}.{schema_name}.{table_name}"

raw_df = spark.sql(query)

raw_df.printSchema()

##Start Update Here 
#Update cast for malformed data for PointNumber column
df = (
    raw_df
    .withColumn("PointNumber", col("PointNumber").cast("int"))
)


df.printSchema()
# df.count()
# display(df.limit(10))


In [0]:
first_point_df = df.filter(df.PointNumber == 1)

display(first_point_df.limit(10))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

window_spec = Window.partitionBy('match_id').orderBy('PointNumber'.desc())

rank_df = df_with_rank = df.withColumn('point_rank', row_number().over(window_spec))

last_point_df = rank_df.filter('point_rank = 1')

match_winner_df = (
    last_point_df
    .select(
        'match_id',
        last_point_df.PointWinner.alias('match_winner')
    )
)

display(match_winner_df.limit(10))
